In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [423 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https://developer.download

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-23 13:32:40--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-05-23 13:32:40 (11.7 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [9]:
preds_df = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)
#county_df = spark.read.jdbc(url=jdbc_url, table='counties', properties=config)
preds_df.show()

+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+------------+-----------+-------------+------------+--------------+------------+-------------+
|  zip|lat|lng|          city|state_id|state_name|  county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|mer_success|rose_success|syr_success|savyb_success|ries_success|cabsav_success|char_success|pinot_success|
+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+------------+-----------+-------------+------------+--------------+------------+-------------+
|50515| 43|-95|      Ayrshire|      IA|      Iowa|    Palo Alto|263.4875000000000...|1013.162500000000...|80.39500000000000...|0.015000000000000000|        0.0|         0.0|        0.0|          0.0|         0.0|           0.0|        

In [10]:
preds_df.dtypes

[('zip', 'bigint'),
 ('lat', 'bigint'),
 ('lng', 'bigint'),
 ('city', 'string'),
 ('state_id', 'string'),
 ('state_name', 'string'),
 ('county_name', 'string'),
 ('avg_temp', 'decimal(38,18)'),
 ('avg_airp', 'decimal(38,18)'),
 ('avg_humid', 'decimal(38,18)'),
 ('avg_precip', 'decimal(38,18)'),
 ('mer_success', 'double'),
 ('rose_success', 'double'),
 ('syr_success', 'double'),
 ('savyb_success', 'double'),
 ('ries_success', 'double'),
 ('cabsav_success', 'double'),
 ('char_success', 'double'),
 ('pinot_success', 'double')]

In [108]:
from pyspark.sql.functions import col,expr


In [14]:
preds_df.columns

['zip',
 'lat',
 'lng',
 'city',
 'state_id',
 'state_name',
 'county_name',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip',
 'mer_success',
 'rose_success',
 'syr_success',
 'savyb_success',
 'ries_success',
 'cabsav_success',
 'char_success',
 'pinot_success']

In [111]:
usepreds = preds_df.select('zip',
 'lat',
 'lng',
 'city',
 'state_id',
 'state_name',
 'county_name',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip',
 'mer_success',
 'syr_success',
 'savyb_success',
 'ries_success',
 'cabsav_success',
 'char_success',
 'pinot_success')

In [112]:
usepreds.show()

+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------+------------+--------------+------------+-------------+
|  zip|lat|lng|          city|state_id|state_name|  county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|mer_success|syr_success|savyb_success|ries_success|cabsav_success|char_success|pinot_success|
+-----+---+---+--------------+--------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------+------------+--------------+------------+-------------+
|50515| 43|-95|      Ayrshire|      IA|      Iowa|    Palo Alto|263.4875000000000...|1013.162500000000...|80.39500000000000...|0.015000000000000000|        0.0|        0.0|          0.0|         0.0|           0.0|         0.0|          0.0|
|50519| 43|-94|          Bode|  

In [117]:
calcpreds = usepreds.toPandas()
calcpreds.head()

,zip,lat,lng,city,state_id,state_name,county_name,avg_temp,avg_airp,avg_humid,avg_precip,mer_success,syr_success,savyb_success,ries_success,cabsav_success,char_success,pinot_success
0,50515,43,-95,Ayrshire,IA,Iowa,Palo Alto,263.487500000000000000,1013.162500000000000000,80.395000000000000000,0.015000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50519,43,-94,Bode,IA,Iowa,Humboldt,266.662500000000000000,1019.822500000000000000,74.757500000000000000,0.005000000000000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,50535,42,-95,Early,IA,Iowa,Sac,266.670000000000000000,1020.350000000000000000,75.497500000000000000,0.010000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,50613,43,-92,Cedar Falls,IA,Iowa,Black Hawk,266.177500000000000000,1018.550000000000000000,79.092500000000000000,0.007500000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50622,43,-92,Denver,IA,Iowa,Bremer,266.177500000000000000,1018.550000000000000000,79.092500000000000000,0.007500000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
calcpreds.dtypes

zip                 int64
lat                 int64
lng                 int64
city               object
state_id           object
state_name         object
county_name        object
avg_temp           object
avg_airp           object
avg_humid          object
avg_precip         object
mer_success       float64
syr_success       float64
savyb_success     float64
ries_success      float64
cabsav_success    float64
char_success      float64
pinot_success     float64
dtype: object

In [120]:
len(calcpreds)

3590

In [121]:
calcpreds['goodwines'] = calcpreds.apply(lambda x: sum(x['mer_success':'pinot_success']), axis=1)
calcpreds['goodwines'].value_counts()

0.0    1868
1.0    1118
2.0     406
3.0     132
4.0      64
5.0       2
Name: goodwines, dtype: int64

In [122]:
calcpreds['pinot_success'].value_counts()

0.0    3022
1.0     568
Name: pinot_success, dtype: int64

In [123]:
sum(calcpreds['pinot_success'])

568.0

In [125]:
champ_counties = calcpreds[calcpreds['goodwines'] == 5]
champ_counties.head()

,zip,lat,lng,city,state_id,state_name,county_name,avg_temp,avg_airp,avg_humid,avg_precip,mer_success,syr_success,savyb_success,ries_success,cabsav_success,char_success,pinot_success,goodwines
1634,94022,37,-122,Los Altos,CA,California,Santa Clara,281.630000000000000000,1019.210000000000000000,73.970000000000000000,0.087500000000000000,1.0,1.0,0.0,0.0,1.0,1.0,1.0,5.0
3522,94022,37,-122,Los Altos,CA,California,Santa Clara,281.630000000000000000,1019.210000000000000000,73.970000000000000000,0.087500000000000000,1.0,1.0,0.0,0.0,1.0,1.0,1.0,5.0


In [126]:
secdplace_counties = calcpreds[calcpreds['goodwines'] == 4]
secdplace_counties['state_id'].value_counts()

GA    40
TN    12
SC     8
AL     2
MS     2
Name: state_id, dtype: int64

In [127]:
thrplace_counties = calcpreds[calcpreds['goodwines'] == 3]
thrplace_counties['state_id'].value_counts()

GA    50
TX    22
AL    16
WI     8
AR     6
CA     6
WA     6
MI     4
IL     4
MO     4
TN     2
MS     2
KY     2
Name: state_id, dtype: int64

In [132]:
def statecalc(bigdf, places):
  statewins = []
  for place in places:
    winvars = 0
    mer = 0
    syr = 0
    savyb = 0
    ries = 0
    cabsav = 0
    char = 0
    pinot = 0
    state_df = bigdf[bigdf['state_name'] == place]
    winvars = sum(state_df['goodwines'])
    mer = sum(state_df['mer_success'])
    syr = sum(state_df['syr_success'])
    savyb = sum(state_df['savyb_success'])
    ries = sum(state_df['ries_success'])
    cabsav = sum(state_df['cabsav_success'])
    char = sum(state_df['char_success'])
    pinot = sum(state_df['pinot_success'])
    statewins.append([place, winvars, mer, syr, savyb, ries, cabsav, char, pinot])
  return pd.DataFrame(statewins, columns=['state','tot_success','merlot','syrah','savyb','riesling','cabsav','chard','pinot'])

In [134]:
statecalc(calcpreds, calcpreds['state_name'].unique())

,state,tot_success,merlot,syrah,savyb,riesling,cabsav,chard,pinot
0,Iowa,18.0,0.0,0.0,16.0,0.0,0.0,0.0,2.0
1,Massachusetts,28.0,0.0,0.0,0.0,10.0,0.0,0.0,18.0
2,Rhode Island,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,New Hampshire,14.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0
4,Wisconsin,98.0,0.0,0.0,0.0,22.0,44.0,32.0,0.0
5,Minnesota,38.0,0.0,0.0,0.0,32.0,6.0,0.0,0.0
6,Maine,6.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0
7,Vermont,10.0,0.0,0.0,0.0,0.0,4.0,0.0,6.0
8,Connecticut,4.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
9,New Jersey,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [136]:
sumchart = statecalc(calcpreds, calcpreds['state_name'].unique())
len(sumchart)

48

In [140]:
sumchart.columns

Index(['state', 'tot_success', 'merlot', 'syrah', 'savyb', 'riesling',
       'cabsav', 'chard', 'pinot'],
      dtype='object')

In [145]:
sumchart.sort_values('tot_success')

,state,tot_success,merlot,syrah,savyb,riesling,cabsav,chard,pinot
11,South Dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,Delaware,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Connecticut,4.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
42,New Mexico,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
37,Wyoming,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,Rhode Island,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
41,Arizona,6.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0
6,Maine,6.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0
9,New Jersey,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
30,Nevada,8.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0


In [138]:
sumchart.to_csv('state_sums.csv')

In [139]:
sumchart.to_json('state_sums.json')